In [1]:
#import packages
from dicom2nifti import convert_dir
from dcmrtstruct2nii import list_rt_structs, dcmrtstruct2nii
import radiomics
#from radiomics import featureextractor  
import pandas as pd
import os
from os import path
import yaml
import pathlib
import pydicom
from pydicom.data import get_testdata_file
import nibabel
import SimpleITK as sitk
#yaml packages
from copy import deepcopy
# Yaml loaders and dumpers
from ruamel.yaml.main import \
    round_trip_load as yaml_load, \
    round_trip_dump as yaml_dump
# Yaml commentary
from ruamel.yaml.comments import \
    CommentedMap as OrderedDict, \
    CommentedSeq as OrderedList
# For manual creation of tokens
from ruamel.yaml.tokens import CommentToken
from ruamel.yaml.error import CommentMark
# Number of spaces for an indent 
INDENTATION = 2 
# Used to reset comment objects
tsRESET_COMMENT_LIST = [None, [], None, None]
##GUI APP
import PySimpleGUI as sg


Please cite:
Thomas Phil, Thomas Albrecht, Skylar Gay, & Mathis Ersted Rasmussen. (2023). Sikerdebaard/dcmrtstruct2nii: dcmrtstruct2nii v5 (Version v5). Zenodo. https://doi.org/10.5281/zenodo.4037864



In [2]:
import collections
import json
import logging
import six

import pykwalify.core
from itertools import chain

from radiomics import generalinfo, getFeatureClasses, getImageTypes, getParameterValidationFiles, imageoperations

logger = logging.getLogger(__name__)
geometryTolerance = None


class RadiomicsFeatureExtractor:

    def __init__(self, *args, **kwargs):
        global logger

        self.settings = {}
        self.enabledImagetypes = {}
        self.enabledFeatures = {}

        self.featureClassNames = list(getFeatureClasses().keys())

        if len(args) == 1 and isinstance(args[0], dict):
            logger.info("Loading parameter dictionary")
            self._applyParams(paramsDict=args[0])
        elif len(args) == 1 and (isinstance(args[0], six.string_types) or isinstance(args[0], pathlib.PurePath)):
            if not os.path.isfile(args[0]):
                raise IOError("Parameter file %s does not exist." % args[0])
            logger.info("Loading parameter file %s", str(args[0]))
            self._applyParams(paramsFile=args[0])
        else:
            # Set default settings and update with and changed settings contained in kwargs
            self.settings = self._getDefaultSettings()
            logger.info('No valid config parameter, using defaults: %s', self.settings)

            self.enabledImagetypes = {'Original': {}}
            logger.info('Enabled image types: %s', self.enabledImagetypes)

            for featureClassName in self.featureClassNames:
                if featureClassName == 'shape2D':  # Do not enable shape2D by default
                    continue
                self.enabledFeatures[featureClassName] = []
                logger.info('Enabled features: %s', self.enabledFeatures)

        if len(kwargs) > 0:
            logger.info('Applying custom setting overrides: %s', kwargs)
            self.settings.update(kwargs)
            logger.debug("Settings: %s", self.settings)

        if self.settings.get('binCount', None) is not None:
            logger.warning('Fixed bin Count enabled! However, we recommend using a fixed bin Width. See '
                           'http://pyradiomics.readthedocs.io/en/latest/faq.html#radiomics-fixed-bin-width for more '
                           'details')

        self._setTolerance()

    def _setTolerance(self):
        global geometryTolerance, logger
        geometryTolerance = self.settings.get('geometryTolerance')
        if geometryTolerance is not None:
            logger.debug('Setting SimpleITK tolerance to %s', geometryTolerance)
            sitk.ProcessObject.SetGlobalDefaultCoordinateTolerance(geometryTolerance)
            sitk.ProcessObject.SetGlobalDefaultDirectionTolerance(geometryTolerance)

    def addProvenance(self, provenance_on=True):
        """
        """
        self.settings['additionalInfo'] = provenance_on

    @staticmethod
    def _getDefaultSettings():
        """
        Returns a dictionary containing the default settings specified in this class. These settings cover global settings,
        such as ``additionalInfo``, as well as the image pre-processing settings (e.g. resampling). Feature class specific
        are defined in the respective feature classes and and not included here. Similarly, filter specific settings are
        defined in ``imageoperations.py`` and also not included here.
        """
        return {'minimumROIDimensions': 2,
                'minimumROISize': None,  # Skip testing the ROI size by default
                'normalize': False,
                'normalizeScale': 1,
                'removeOutliers': None,
                'resampledPixelSpacing': None,  # No resampling by default
                'interpolator': 'sitkBSpline',  # Alternative: sitk.sitkBSpline
                'preCrop': False,
                'padDistance': 5,
                'distances': [1],
                'force2D': False,
                'force2Ddimension': 0,
                'resegmentRange': None,  # No resegmentation by default
                'label': 1,
                'additionalInfo': True}

    def loadParams(self, paramsFile):
        """
        Parse specified parameters file and use it to update settings, enabled feature(Classes) and image types. For more
        information on the structure of the parameter file, see
        :ref:`Customizing the extraction <radiomics-customization-label>`.

        If supplied file does not match the requirements (i.e. unrecognized names or invalid values for a setting), a
        pykwalify error is raised.
        """
        self._applyParams(paramsFile=paramsFile)

    def loadJSONParams(self, JSON_configuration):
        """
        """
        parameter_data = json.loads(JSON_configuration)
        self._applyParams(paramsDict=parameter_data)

    def _applyParams(self, paramsFile=None, paramsDict=None):
        """
        """
        global logger

        # Ensure pykwalify.core has a log handler (needed when parameter validation fails)
        if len(pykwalify.core.log.handlers) == 0 and len(logging.getLogger().handlers) == 0:
            # No handler available for either pykwalify or root logger, provide first radiomics handler (outputs to stderr)
            pykwalify.core.log.addHandler(logging.getLogger('radiomics').handlers[0])

        schemaFile, schemaFuncs = getParameterValidationFiles()
        c = pykwalify.core.Core(source_file=paramsFile, source_data=paramsDict,
                                schema_files=[schemaFile], extensions=[schemaFuncs])
        params = c.validate()
        logger.debug('Parameters parsed, input is valid.')

        enabledImageTypes = params.get('imageType', {})
        enabledFeatures = params.get('featureClass', {})
        settings = params.get('setting', {})
        voxelSettings = params.get('voxelSetting', {})

        logger.debug("Applying settings")

        if len(enabledImageTypes) == 0:
            self.enabledImagetypes = {'Original': {}}
        else:
            self.enabledImagetypes = enabledImageTypes

        logger.debug("Enabled image types: %s", self.enabledImagetypes)

        if len(enabledFeatures) == 0:
            self.enabledFeatures = {}
            for featureClassName in self.featureClassNames:
                self.enabledFeatures[featureClassName] = []
        else:
            self.enabledFeatures = enabledFeatures

        logger.debug("Enabled features: %s", self.enabledFeatures)

        # Set default settings and update with and changed settings contained in kwargs
        self.settings = self._getDefaultSettings()
        self.settings.update(settings)
        self.settings.update(voxelSettings)

        logger.debug("Settings: %s", settings)

    def execute(self, imageFilepath, maskFilepath, label=None, label_channel=None, voxelBased=False):
        """
        
        """
        global geometryTolerance, logger
        _settings = self.settings.copy()

        tolerance = _settings.get('geometryTolerance')
        additionalInfo = _settings.get('additionalInfo', False)
        resegmentShape = _settings.get('resegmentShape', False)

        if label is not None:
            _settings['label'] = label
        else:
            label = _settings.get('label', 1)

        if label_channel is not None:
            _settings['label_channel'] = label_channel

        if geometryTolerance != tolerance:
            self._setTolerance()

        if additionalInfo:
            generalInfo = generalinfo.GeneralInfo()
            generalInfo.addGeneralSettings(_settings)
            generalInfo.addEnabledImageTypes(self.enabledImagetypes)
        else:
            generalInfo = None

        if voxelBased:
            _settings['voxelBased'] = True
            kernelRadius = _settings.get('kernelRadius', 1)
            logger.info('Starting voxel based extraction')
        else:
            kernelRadius = 0

        logger.info('Calculating features with label: %d', label)
        logger.debug('Enabled images types: %s', self.enabledImagetypes)
        logger.debug('Enabled features: %s', self.enabledFeatures)
        logger.debug('Current settings: %s', _settings)

        # 1. Load the image and mask
        featureVector = collections.OrderedDict()
        image, mask = self.loadImage(imageFilepath, maskFilepath, generalInfo, **_settings)

        # 2. Check whether loaded mask contains a valid ROI for feature extraction and get bounding box
        # Raises a ValueError if the ROI is invalid
        boundingBox, correctedMask = imageoperations.checkMask(image, mask, **_settings)

        # Update the mask if it had to be resampled
        if correctedMask is not None:
            if generalInfo is not None:
                generalInfo.addMaskElements(image, correctedMask, label, 'corrected')
            mask = correctedMask

        logger.debug('Image and Mask loaded and valid, starting extraction')

        # 5. Resegment the mask if enabled (parameter regsegmentMask is not None)
        resegmentedMask = None
        if _settings.get('resegmentRange', None) is not None:
            resegmentedMask = imageoperations.resegmentMask(image, mask, **_settings)

            # Recheck to see if the mask is still valid, raises a ValueError if not
            boundingBox, correctedMask = imageoperations.checkMask(image, resegmentedMask, **_settings)

            if generalInfo is not None:
                generalInfo.addMaskElements(image, resegmentedMask, label, 'resegmented')

        # 3. Add the additional information if enabled
        if generalInfo is not None:
            featureVector.update(generalInfo.getGeneralInfo())

        # if resegmentShape is True and resegmentation has been enabled, update the mask here to also use the
        # resegmented mask for shape calculation (e.g. PET resegmentation)
        if resegmentShape and resegmentedMask is not None:
            mask = resegmentedMask

        if not voxelBased:
            # 4. If shape descriptors should be calculated, handle it separately here
            featureVector.update(self.computeShape(image, mask, boundingBox, **_settings))

        # (Default) Only use resegemented mask for feature classes other than shape
        # can be overridden by specifying `resegmentShape` = True
        if not resegmentShape and resegmentedMask is not None:
            mask = resegmentedMask

        # 6. Calculate other enabled feature classes using enabled image types
        # Make generators for all enabled image types
        logger.debug('Creating image type iterator')
        imageGenerators = []
        for imageType, customKwargs in six.iteritems(self.enabledImagetypes):
            args = _settings.copy()
            args.update(customKwargs)
            logger.info('Adding image type "%s" with custom settings: %s' % (imageType, str(customKwargs)))
            imageGenerators = chain(imageGenerators, getattr(imageoperations, 'get%sImage' % imageType)(image, mask,
                                                                                                        **args))

        logger.debug('Extracting features')
        # Calculate features for all (filtered) images in the generator
        for inputImage, imageTypeName, inputKwargs in imageGenerators:
            logger.info('Calculating features for %s image', imageTypeName)
            inputImage, inputMask = imageoperations.cropToTumorMask(inputImage, mask, boundingBox, padDistance=kernelRadius)
            featureVector.update(self.computeFeatures(inputImage, inputMask, imageTypeName, **inputKwargs))

        logger.debug('Features extracted')

        return featureVector

    @staticmethod
    def loadImage(ImageFilePath, MaskFilePath, generalInfo=None, **kwargs):
        """
        
        """
        global logger

        normalize = kwargs.get('normalize', False)
        interpolator = kwargs.get('interpolator')
        resampledPixelSpacing = kwargs.get('resampledPixelSpacing')
        preCrop = kwargs.get('preCrop', False)
        label = kwargs.get('label', 1)

        logger.info('Loading image and mask')
        if isinstance(ImageFilePath, six.string_types) and os.path.isfile(ImageFilePath):
            image = sitk.ReadImage(ImageFilePath)
        elif isinstance(ImageFilePath, sitk.SimpleITK.Image):
            image = ImageFilePath
        else:
            raise ValueError('Error reading image Filepath or SimpleITK object')

        if isinstance(MaskFilePath, six.string_types) and os.path.isfile(MaskFilePath):
            mask = sitk.ReadImage(MaskFilePath)
        elif isinstance(MaskFilePath, sitk.SimpleITK.Image):
            mask = MaskFilePath
        else:
            raise ValueError('Error reading mask Filepath or SimpleITK object')

        # process the mask
        mask = imageoperations.getMask(mask, **kwargs)

        if generalInfo is not None:
            generalInfo.addImageElements(image)
            # Do not include the image here, as the overlap between image and mask have not been checked
            # It is therefore possible that image and mask do not align, or even have different sizes.
            generalInfo.addMaskElements(None, mask, label)

        # This point is only reached if image and mask loaded correctly
        if normalize:
            image = imageoperations.normalizeImage(image, **kwargs)

        if interpolator is not None and resampledPixelSpacing is not None:
            image, mask = imageoperations.resampleImage(image, mask, **kwargs)
            if generalInfo is not None:
                generalInfo.addImageElements(image, 'interpolated')
                generalInfo.addMaskElements(image, mask, label, 'interpolated')

        elif preCrop:
            bb, correctedMask = imageoperations.checkMask(image, mask, **kwargs)
            if correctedMask is not None:
                # Update the mask if it had to be resampled
                mask = correctedMask
            if bb is None:
                # Mask checks failed
                raise ValueError('Mask checks failed during pre-crop')

            image, mask = imageoperations.cropToTumorMask(image, mask, bb, **kwargs)

        return image, mask

    def computeShape(self, image, mask, boundingBox, **kwargs):
        """
        
        """
        global logger
        featureVector = collections.OrderedDict()

        enabledFeatures = self.enabledFeatures

        croppedImage, croppedMask = imageoperations.cropToTumorMask(image, mask, boundingBox)

        # Define temporary function to compute shape features
        def compute(shape_type):
            logger.info('Computing %s', shape_type)
            featureNames = enabledFeatures[shape_type]
            shapeClass = getFeatureClasses()[shape_type](croppedImage, croppedMask, **kwargs)

            if featureNames is not None:
                for feature in featureNames:
                    shapeClass.enableFeatureByName(feature)

            for (featureName, featureValue) in six.iteritems(shapeClass.execute()):
                newFeatureName = 'original_%s_%s' % (shape_type, featureName)
                featureVector[newFeatureName] = featureValue

        Nd = mask.GetDimension()
        if 'shape' in enabledFeatures.keys():
            if Nd == 3:
                compute('shape')
            else:
                logger.warning('Shape features are only available 3D input (for 2D input, use shape2D). Found %iD input',
                               Nd)

        if 'shape2D' in enabledFeatures.keys():
            if Nd == 3:
                force2D = kwargs.get('force2D', False)
                force2Ddimension = kwargs.get('force2Ddimension', 0)
                if not force2D:
                    logger.warning('parameter force2D must be set to True to enable shape2D extraction')
                elif not (boundingBox[1::2] - boundingBox[0::2] + 1)[force2Ddimension] > 1:
                    logger.warning('Size in specified 2D dimension (%i) is greater than 1, cannot calculate 2D shape',
                                   force2Ddimension)
                else:
                    compute('shape2D')
            elif Nd == 2:
                compute('shape2D')
            else:
                logger.warning('Shape2D features are only available for 2D and 3D (with force2D=True) input. '
                               'Found %iD input', Nd)

        return featureVector

    def computeFeatures(self, image, mask, imageTypeName, **kwargs):
        r"""
        
        """
        global logger
        featureVector = collections.OrderedDict()
        featureClasses = getFeatureClasses()

        enabledFeatures = self.enabledFeatures

        # Calculate feature classes
        for featureClassName, featureNames in six.iteritems(enabledFeatures):
            # Handle calculation of shape features separately
            if featureClassName.startswith('shape'):
                continue

            if featureClassName in featureClasses:
                logger.info('Computing %s', featureClassName)

                featureClass = featureClasses[featureClassName](image, mask, **kwargs)

                if featureNames is not None:
                    for feature in featureNames:
                        featureClass.enableFeatureByName(feature)

                for (featureName, featureValue) in six.iteritems(featureClass.execute()):
                    newFeatureName = '%s_%s_%s' % (imageTypeName, featureClassName, featureName)
                    featureVector[newFeatureName] = featureValue

        return featureVector

    def enableAllImageTypes(self):
        """
        Enable all possible image types without any custom settings.
        """
        global logger

        logger.debug('Enabling all image types')
        for imageType in getImageTypes():
            self.enabledImagetypes[imageType] = {}
        logger.debug('Enabled images types: %s', self.enabledImagetypes)

    def disableAllImageTypes(self):
        """
        
        """
        global logger

        logger.debug('Disabling all image types')
        self.enabledImagetypes = {}

    def enableImageTypeByName(self, imageType, enabled=True, customArgs=None):
        r"""
        Enable or disable specified image type. If enabling image type, optional custom settings can be specified in
        customArgs.

        
        """
        global logger

        if imageType not in getImageTypes():
            logger.warning('Image type %s is not recognized', imageType)
            return

        if enabled:
            if customArgs is None:
                customArgs = {}
                logger.debug('Enabling image type %s (no additional custom settings)', imageType)
            else:
                logger.debug('Enabling image type %s (additional custom settings: %s)', imageType, customArgs)
            self.enabledImagetypes[imageType] = customArgs
        elif imageType in self.enabledImagetypes:
            logger.debug('Disabling image type %s', imageType)
            del self.enabledImagetypes[imageType]
        logger.debug('Enabled images types: %s', self.enabledImagetypes)

    def enableImageTypes(self, **enabledImagetypes):
        
        global logger

        logger.debug('Updating enabled images types with %s', enabledImagetypes)
        self.enabledImagetypes.update(enabledImagetypes)
        logger.debug('Enabled images types: %s', self.enabledImagetypes)

    def enableAllFeatures(self):
        
        global logger

        logger.debug('Enabling all features in all feature classes')
        for featureClassName in self.featureClassNames:
            self.enabledFeatures[featureClassName] = []
        logger.debug('Enabled features: %s', self.enabledFeatures)

    def disableAllFeatures(self):
        """
        Disable all classes.
        """
        global logger

        logger.debug('Disabling all feature classes')
        self.enabledFeatures = {}

    def enableFeatureClassByName(self, featureClass, enabled=True):
        
        global logger

        if featureClass not in self.featureClassNames:
            logger.warning('Feature class %s is not recognized', featureClass)
            return

        if enabled:
            logger.debug('Enabling all features in class %s', featureClass)
            self.enabledFeatures[featureClass] = []
        elif featureClass in self.enabledFeatures:
            logger.debug('Disabling feature class %s', featureClass)
            del self.enabledFeatures[featureClass]
        logger.debug('Enabled features: %s', self.enabledFeatures)

    def enableFeaturesByName(self, **enabledFeatures):
        
        global logger

        logger.debug('Updating enabled features with %s', enabledFeatures)
        self.enabledFeatures.update(enabledFeatures)
        logger.debug('Enabled features: %s', self.enabledFeatures)

ImportError: cannot import name 'generalinfo' from 'radiomics' (C:\Users\bertolinima\AppData\Roaming\Python\Python312\site-packages\radiomics\__init__.py)

In [ ]:
#convert DICOM files in NIFTI
#path = percorso della cartella contenente i pazienti. Ogni cartella del pz deve avere una sottocartella con le immagini
#e una cartella con le RT struct

def dicom2nifti(path,mod):
    c=0
    
    patients = os.listdir(path)
    for i in patients:
        
        patient_path = path+"/"+i
        
        for j in mod:
            
            image_path = patient_path+"/"+j
            
            file_nii = pathlib.Path(patient_path+"/mask_"+j+"/"+"nii_image")
            file = pathlib.Path(patient_path+"/mask_"+j)

            if file.exists()==False:
                os.mkdir(patient_path+"/mask_"+j)
            
            new_dir = patient_path+"/mask_"+j
            
            if file_nii.exists()==False:
                os.mkdir(patient_path+"/mask_"+j+"/"+"nii_image")
                
            new_dir_nii = patient_path+"/mask_"+j+"/"+"nii_image"
  
            convert_dir.convert_directory(image_path,new_dir_nii,compression=False,reorient=False)
    
            if j=="PET":
                files = [each for each in os.listdir(new_dir_nii+"/") if each.endswith('.nii')]

                tac=nibabel.load(new_dir_nii+"/"+files[0])
                dicom = os.listdir(image_path)
  
                ds = pydicom.dcmread(image_path+"/"+dicom[1])
               
                if ds.PatientWeight =='' or ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose=='':
                    sg.popup_error('ERROR! Weight and Activity Tag DICOM empty')
                    activity=sg.popup_get_text('Insert Activity value',title="Activity value")
                    peso = sg.popup_get_text('Insert weight value',title="weight value")
                    array=tac.get_fdata()
                    a=float(activity)
                    w=float(peso)*1000
                    array=array/(a/w)
                    new_tac=nibabel.Nifti1Image(array,tac.affine,tac.header)
                    nibabel.save(new_tac,new_dir_nii)
                    os.remove(new_dir_nii+"/"+files[0])
                else:
                    peso=ds.PatientWeight
                    activity=ds.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose
                    array=tac.get_fdata()
                    a=float(activity)
                    w=float(peso)
                    array=array/(a/w)
                    new_tac=nibabel.Nifti1Image(array,tac.affine,tac.header)
                    nibabel.save(new_tac,new_dir_nii+"/"+"new_nifti_image")
                    #os.remove(new_dir_nii+"/"+files[0])
                
                
                
            structures = os.listdir(patient_path+"/"+j+"_SR")
            str_path = patient_path+"/"+j+"_SR"+"/"+structures[0]
            structs = list_rt_structs(str_path)
            files = [each for each in os.listdir(patient_path+"/") if each.endswith('.nii')]
        
   
            dcmrtstruct2nii(str_path,image_path,new_dir,gzip=False,convert_original_dicom=False,structures=structs)
    
            window['PBAR_dicom'].update(current_count=c)
            window['OUT_dicom'].update(str(i)+str(mod))
            c=c+1

In [ ]:
def control_function(path,mod):
    
    patient = os.listdir(path)
    file_exist=[]
    file_exist_mod=[]
    for i in patient:
        for j in mod:
            patt = path+"/"+i+"/mask_"+j
            
            file = os.path.exists(patt)
            file_nii = os.path.exists(patt+"/nii_image")
            if file==True and file_nii==True:
                mask = os.listdir(patt)
                nii = os.listdir(patt+"/nii_image") 
                if len(mask)>=1 and len(nii)>=1:
                    file_exist_mod.append(j)
                    
        if len(file_exist_mod)==len(mod):
            file_exist.append(i)
            file_exist_mod.clear()
    return file_exist

In [ ]:
#generate yaml file
def yamlfile(norm,norm_scale,outli,pixel_space,interp,precrop,res,res_mode,binn,two_d,force_dim):
 
        paramFile ={
            "imageType":{
                "Original": {},
                "LoG": {'sigma' : [1.0, 3.0]},
                "Wavelet": {},
                "Square": {},
                "SquareRoot": {},
                "Logarithm": {},
                "Exponential":{},
                "Gradient":{},
                },
            "setting": {
                    "normalize":norm,
                    "normalizeScale":norm_scale,
                    "removeOutliers":outli,
                    "resampledPixelSpacing":[pixel_space[0],pixel_space[1],pixel_space[2]],
                    "interpolator":interp,
                    "preCrop": precrop,
                    "resegmentRange": res,
                    "resegmentMode":res_mode,
                    "binWidth": binn, 
                    "force2D":two_d,
                    "force2Ddimension":force_dim,
                    
                    
            
                }
            }

     
        
        return paramFile

In [ ]:
def GUI_POPUP(text, data):
    value=[]
    layout = [
        [sg.Text(text)],
        [sg.Listbox(data, size=(20,5), key='SELECTED',select_mode="multiple")],
        [sg.Button('OK')],
    ]
    
    window = sg.Window('POPUP', layout).Finalize()
    
    while True:
        event, values = window.read()

        if event == sg.WINDOW_CLOSED or event == 'OK':
            if len(values['SELECTED'])==0:
                sg.popup_error("Devi scegliere una struttura")
                
                #value.append(values['SELECTED'])
            break
        
        

    window.close()
    return values['SELECTED']

In [ ]:
def radiomics(param_file,pat_folder,modality):
    results_tot=[]
    df_tot = pd.DataFrame()
    path = pat_folder+"/"
    patients = os.listdir(path)
    results = pd.DataFrame()
    c=0
    counter=0
    size_pbar=len(patients)
    window['PBAR'].update(current_count= 0, max=size_pbar)
    prova = sg.popup_yes_no("Do you want to select the ROI?")
    for i in patients:
        
        patient_path = path+i
            
        extractor = RadiomicsFeatureExtractor(param_file)

        #file_nii = pathlib.Path(patient_path+"/mask_"+j+"/"+"nii_image")
        file_nii = patient_path+"/mask_"+modality+"/"+"nii_image"
        #struct_nii = pathlib.Path(patient_path+"/mask_"+j)
        struct_nii = patient_path+"/mask_"+modality
        if modality=="PET":
            nii_images_path = file_nii+"/"+"new_nifti_image.nii"
        else:
                            
            nii_images = [each for each in os.listdir(file_nii+"/") if each.endswith('.nii')]
            nii_images_path = file_nii+"/"+nii_images[0]
   
        
        structures_1 = [each for each in os.listdir(struct_nii+"/") if each.endswith('.nii')]
        
        if prova == "Yes":
            structures = GUI_POPUP("scegli la struttura", structures_1)
        else:
            structures = strucutures_1
            
        size_pbar=len(structures)+len(patients)
        window['PBAR'].update(current_count= 0, max=size_pbar)
        for struct in structures:
            counter += 1
            result=extractor.execute(nii_images_path,struct_nii+"/"+struct,label=255)
            
            results= pd.DataFrame.from_dict(result, orient='index', dtype=None,columns = None)
            results_1=results.iloc[47:,:].transpose()
            
            results_1["Patient"]=i
            results_1["Modality"]=modality
            results_1["Structure"]=struct
            
            results_tot.append(results_1)
            window['PBAR'].Update(counter)
            
            window['-OUT-'].Update("patient: "+str(i)+" Modality: "+str(mod)+" Struct: "+str(struct))
            window.refresh()
            if c==0: 
                df_tot=pd.concat(results_tot)
            else:
                df_tot=pd.concat(results_tot)
            c=c+1       
    window['-OUT-'].update("finish")   
    return df_tot

In [ ]:
sg.theme('GreenMono')  # please make your windows colorful


t_f = ["yes","no"]
mod=[]
yaml_da_fuori=[]
interpolators=["sitkNearestNeighbor","sitkLinear","sitkBSpline","sitkGaussian","sitkLabelGaussian","sitkHammingWindowedSinc",
               "sitkCosineWindowedSinc","sitkWelchWindowedSinc","sitkLanczosWindowedSinc","sitkBlackmanWindowedSinc"]

layout = [[sg.Text("Images modality"),sg.Checkbox("CT",key="CT",change_submits=True,enable_events=True),sg.Checkbox("CT/CT-PET",key="CT/CT-PET",change_submits=True,enable_events=True),
           sg.Checkbox("PET",key="PET",change_submits=True,enable_events=True),sg.Checkbox("MR",key="MR",change_submits=True,enable_events=True)],
          [sg.Text("Choose patients folder: "), sg.Input(key="-IN2-" ,change_submits=True), sg.FolderBrowse(key="-IN-",change_submits=True)],
          [sg.Button('Convert to NIFTI')],
          [sg.ProgressBar(100, orientation='h', expand_x=True, size=(20, 20),  key='PBAR_dicom',bar_color='yellow')],
          [sg.Text('', key='OUT_dicom', enable_events=True, font=('Arial Bold', 12), expand_x=True)],
          [sg.Button("Proceed with radiomics",key="radiomic_proceed")],
          [sg.Text("Folder where to save the radiomic file",key="text_radiomic_save",visible=False),sg.Input("",key="input_radiomic_save",visible=False,size=(15,1)),
           sg.FolderBrowse("radiomic save folder",key="radiomic_save",change_submits=True,enable_events=True,visible=False)],
          [sg.Button("Radiomic",visible=False,key="radiomic"),sg.StatusBar("",size=(0,1),key="radiomic_status",visible=False)],
          [sg.ProgressBar(100, orientation='h', expand_x=True, size=(20, 20),  key='PBAR',visible=False,bar_color='yellow')],
          [sg.Text('', key='-OUT-', enable_events=True, font=('Arial Bold', 12), expand_x=True,visible=False)],
          [sg.Button("Reset",visible=False,key="reset")]]


layout_ct = [[sg.Text("Create new yaml file?",key="y text_ct",visible=True),sg.Combo(t_f,enable_events=True, readonly=False, key='new_yaml_ct',visible=True),
           sg.Text("Choose your file",visible=False,key="yaml_da_fuori_text_ct")],
           [sg.FileBrowse("CT Param File",key="yaml_CT",visible=False,file_types=[("YAML Files","*.yaml")]),sg.Input(key="yaml_CT_input",visible=False)],
           [sg.Text("Normalize image",key="norm_text_ct",visible=True), sg.Checkbox('Enable',key="norm_ct" ,change_submits=True,enable_events=True,visible=True),
            sg.Text("Normalize scale ",size=(13,1),key="text_scale_ct",visible=False), 
           sg.Input(key="scale_ct" ,tooltip="Insert number>0",change_submits=True,visible=False,size=(5,1)),
           sg.Text("Remove outliers",key="text out_ct",visible=False),
            sg.Combo(t_f,enable_events=True, readonly=False, key='outliers_ct',visible=False),
           sg.Input(key="out_scale_ct" ,change_submits=True,visible=False,size=(15,1))],
          [sg.Text("Resampled Pixel Spacing:",key="res_text_ct",visible=True),sg.Input("x",key="res_space_1_ct" ,change_submits=True,size=(5,1),visible=True),
           sg.Input("y",key="res_space_2_ct" ,change_submits=True,size=(5,1),visible=True),sg.Input("z",key="res_space_3_ct" ,change_submits=True,size=(5,1),visible=True)],
          [sg.Text("Interpolator",key="int_text_ct",visible=True),sg.Combo(interpolators,enable_events=True, readonly=False, key='interp_ct',default_value="sitkBSpline",visible=True)],
          [sg.Text("Precrop",key="precrop_text_ct",visible=True),
            sg.Checkbox('Enable',key="precrop_ct" ,change_submits=True,enable_events=True,visible=True)],
          [sg.Text("Resegment Range",key="reseg_text_ct",visible=True), sg.Checkbox('Enable',key="res_range_ct" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Lower value",key="lower_text_ct",size=(10,1),visible=False),
           sg.Input(key="lower_ct" ,change_submits=True,size=(5,1),visible=False),sg.Text("Upper value (opt.)",key="upper_text_ct",size=(15,1),visible=False),
           sg.Input(0,key="upper_ct" ,change_submits=True,size=(5,1),visible=False)],
          [sg.Text("Resegment mode",key="text_mode_ct",visible=False),sg.Combo(["absolute","relative","sigma (3std)"],key="res_mode_ct",visible=False,default_value="absolute")],
          [sg.Text("BinWidth",key="bin_text_ct",visible=True),sg.Input(key="bin_w_ct",size=(5,1),visible=True)],
          [sg.Text("Force 2D extraction",key="force_text_ct",visible=True),sg.Checkbox('Enable',key="force_ct" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Force 2D dimension (Btwn [0,2])",key="force_dim_text_ct",visible=False,size=(25,1)),sg.Input(key="force_dim_ct",size=(5,1),visible=False)],
          [sg.Text("Choose your save folder: ",key="choose_folder_text_ct",visible=True), sg.FolderBrowse(key="-IN4-_ct",visible=True)],
          [sg.Button("Generate yaml File",key="generate_param_ct",visible=True)]]


layout_ctpet = [[sg.Text("Create new yaml file?",key="y text_ctpet",visible=True),sg.Combo(t_f,enable_events=True, readonly=False, key='new_yaml_ctpet',visible=True),
           sg.Text("Choose your file_ctpet",visible=False,key="yaml_da_fuori_text_ctpet")],
          [sg.FileBrowse("CTPET Param File",key="yaml_CTPET",visible=False,file_types=[("YAML Files","*.yaml")]),sg.Input(key="yaml_CTPET_input",visible=False)],
           [sg.Text("Normalize image",key="norm_text_ctpet",visible=True), sg.Checkbox('Enable',key="norm_ctpet" ,change_submits=True,enable_events=True,visible=True),
            sg.Text("Normalize scale ",size=(13,1),key="text_scale_ctpet",visible=False), 
           sg.Input(key="scale_ctpet" ,tooltip="Insert number>0",change_submits=True,visible=False,size=(5,1)),
           sg.Text("Remove outliers",key="text out_ctpet",visible=False),
            sg.Combo(t_f,enable_events=True, readonly=False, key='outliers_ctpet',visible=False),
           sg.Input(key="out_scale_ctpet" ,change_submits=True,visible=False,size=(15,1))],
          [sg.Text("Resampled Pixel Spacing:",key="res_text_ctpet",visible=True),sg.Input("x",key="res_space_1_ctpet" ,change_submits=True,size=(5,1),visible=True),
           sg.Input("y",key="res_space_2_ctpet" ,change_submits=True,size=(5,1),visible=True),sg.Input("z",key="res_space_3_ctpet" ,change_submits=True,size=(5,1),visible=True)],
          [sg.Text("Interpolator",key="int_text_ctpet",visible=True),sg.Combo(interpolators,enable_events=True, readonly=False, key='interp_ctpet',default_value="sitkBSpline",visible=True)],
          [sg.Text("Precrop",key="precrop_text_ctpet",visible=True),
            sg.Checkbox('Enable',key="precrop_ctpet" ,change_submits=True,enable_events=True,visible=True)],
          [sg.Text("Resegment Range",key="reseg_text_ctpet",visible=True), sg.Checkbox('Enable',key="res_range_ctpet" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Lower value",key="lower_text_ctpet",size=(10,1),visible=False),
           sg.Input(key="lower_ctpet" ,change_submits=True,size=(5,1),visible=False),sg.Text("Upper value (opt.)",key="upper_text_ctpet",size=(15,1),visible=False),
           sg.Input(0,key="upper_ctpet" ,change_submits=True,size=(5,1),visible=False)],
          [sg.Text("Resegment mode",key="text_mode_ctpet",visible=False),sg.Combo(["absolute","relative","sigma (3std)"],key="res_mode_ctpet",visible=False,default_value="absolute")],
          [sg.Text("BinWidth",key="bin_text_ctpet",visible=True),sg.Input(key="bin_w_ctpet",size=(5,1),visible=True)],
          [sg.Text("Force 2D extraction",key="force_text_ctpet",visible=True),sg.Checkbox('Enable',key="force_ctpet" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Force 2D dimension (Btwn [0,2])",key="force_dim_text_ctpet",visible=False,size=(25,1)),sg.Input(key="force_dim_ctpet",size=(5,1),visible=False)],
          [sg.Text("Choose your save folder: ",key="choose_folder_text_ctpet",visible=True), sg.FolderBrowse(key="-IN4-_ctpet",visible=True)],
          [sg.Button("Generate yaml File",key="generate_param_ctpet",visible=True)]]


layout_pet = [[sg.Text("Create new yaml file?",key="y text_pet",visible=True),sg.Combo(t_f,enable_events=True, readonly=False, key='new_yaml_pet',visible=True),
           sg.Text("Choose your file",visible=False,key="yaml_da_fuori_text_pet")],
          [sg.FileBrowse("PET Param File",key="yaml_PET",visible=False,file_types=[("YAML Files","*.yaml")]),sg.Input(key="yaml_PET_input",visible=False)],
            [sg.Text("Normalize image",key="norm_text_pet",visible=True), sg.Checkbox('Enable',key="norm_pet" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Normalize scale ",size=(13,1),key="text_scale_pet",visible=False), 
           sg.Input(key="scale_pet" ,tooltip="Insert number>0",change_submits=True,visible=False,size=(5,1)),
           sg.Text("Remove outliers",key="text out_pet",visible=False),
            sg.Combo(t_f,enable_events=True, readonly=False, key='outliers_pet',visible=False),
           sg.Input(key="out_scale_pet" ,change_submits=True,visible=False,size=(15,1))],
          [sg.Text("Resampled Pixel Spacing:",key="res_text_pet",visible=True),sg.Input("x",key="res_space_1_pet" ,change_submits=True,size=(5,1),visible=True),
           sg.Input("y",key="res_space_2_pet" ,change_submits=True,size=(5,1),visible=True),sg.Input("z",key="res_space_3_pet" ,change_submits=True,size=(5,1),visible=True)],
          [sg.Text("Interpolator",key="int_text_pet",visible=True),sg.Combo(interpolators,enable_events=True, readonly=False, key='interp_pet',default_value="sitkBSpline",visible=True)],
          [sg.Text("Precrop",key="precrop_text_pet",visible=True),
            sg.Checkbox('Enable',key="precrop_pet" ,change_submits=True,enable_events=True,visible=True)],
          [sg.Text("Resegment Range",key="reseg_text_pet",visible=True), sg.Checkbox('Enable',key="res_range_pet" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Lower value",key="lower_text_pet",size=(10,1),visible=False),
           sg.Input(key="lower_pet" ,change_submits=True,size=(5,1),visible=False),sg.Text("Upper value (opt.)",key="upper_text_pet",size=(15,1),visible=False),
           sg.Input(0,key="upper_pet" ,change_submits=True,size=(5,1),visible=False)],
          [sg.Text("Resegment mode",key="text_mode_pet",visible=False),sg.Combo(["absolute","relative","sigma (3std)"],key="res_mode_pet",visible=False,default_value="absolute")],
          [sg.Text("BinWidth",key="bin_text_pet",visible=True),sg.Input(key="bin_w_pet",size=(5,1),visible=True)],
          [sg.Text("Force 2D extraction",key="force_text_pet",visible=True),sg.Checkbox('Enable',key="force_pet" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Force 2D dimension (Btwn [0,2])",key="force_dim_text_pet",visible=False,size=(25,1)),sg.Input(key="force_dim_pet",size=(5,1),visible=False)],
          [sg.Text("Choose your save folder: ",key="choose_folder_text_pet",visible=True), sg.FolderBrowse(key="-IN4-_pet",visible=True)],
          [sg.Button("Generate yaml File",key="generate_param_pet",visible=True)]]

layout_rm = [[sg.Text("Create new yaml file?",key="y text_rm",visible=True),sg.Combo(t_f,enable_events=True, readonly=False, key='new_yaml_rm',visible=True),
           sg.Text("Choose your file",visible=False,key="yaml_da_fuori_text_rm")],
          [sg.FileBrowse("PET Param File",key="yaml_RM",visible=False,file_types=[("YAML Files","*.yaml")]),sg.Input(key="yaml_RM_input",visible=False)],
            [sg.Text("Normalize image",key="norm_text_rm",visible=True), sg.Checkbox('Enable',key="norm_rm" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Normalize scale ",size=(13,1),key="text_scale_rm",visible=False), 
           sg.Input(key="scale_rm" ,tooltip="Insert number>0",change_submits=True,visible=False,size=(5,1)),
           sg.Text("Remove outliers",key="text out_rm",visible=False),
            sg.Combo(t_f,enable_events=True, readonly=False, key='outliers_rm',visible=False),
           sg.Input(key="out_scale_rm" ,change_submits=True,visible=False,size=(15,1))],
          [sg.Text("Resampled Pixel Spacing:",key="res_text_rm",visible=True),sg.Input("x",key="res_space_1_rm" ,change_submits=True,size=(5,1),visible=True),
           sg.Input("y",key="res_space_2_rm" ,change_submits=True,size=(5,1),visible=True),sg.Input("z",key="res_space_3_rm" ,change_submits=True,size=(5,1),visible=True)],
          [sg.Text("Interpolator",key="int_text_rm",visible=True),sg.Combo(interpolators,enable_events=True, readonly=False, key='interp_rm',default_value="sitkBSpline",visible=True)],
          [sg.Text("Precrop",key="precrop_text_rm",visible=True),
            sg.Checkbox('Enable',key="precrop_rm" ,change_submits=True,enable_events=True,visible=True)],
          [sg.Text("Resegment Range",key="reseg_text_rm",visible=True), sg.Checkbox('Enable',key="res_range_rm" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Lower value",key="lower_text_rm",size=(10,1),visible=False),
           sg.Input(key="lower_rm" ,change_submits=True,size=(5,1),visible=False),sg.Text("Upper value (opt.)",key="upper_text_rm",size=(15,1),visible=False),
           sg.Input(0,key="upper_rm" ,change_submits=True,size=(5,1),visible=False)],
          [sg.Text("Resegment mode",key="text_mode_rm",visible=False),sg.Combo(["absolute","relative","sigma (3std)"],key="res_mode_rm",visible=False,default_value="sigma (3std)")],
          [sg.Text("BinWidth",key="bin_text_rm",visible=True),sg.Input(key="bin_w_rm",size=(5,1),visible=True)],
          [sg.Text("Force 2D extraction",key="force_text_rm",visible=True),sg.Checkbox('Enable',key="force_rm" ,change_submits=True,enable_events=True,visible=True),
           sg.Text("Force 2D dimension (Btwn [0,2])",key="force_dim_text_rm",visible=False,size=(25,1)),sg.Input(key="force_dim_rm",size=(5,1),visible=False)],
          [sg.Text("Choose your save folder: ",key="choose_folder_text_rm",visible=True), sg.FolderBrowse(key="-IN4-_rm",visible=True)],
          [sg.Button("Generate yaml File",key="generate_param_rm",visible=True)]]


tabgrp = [[sg.TabGroup([[sg.Tab('Main window', layout,
                                tooltip='Main window', key= "main"),
                    sg.Tab('CT', layout_ct,title_color='Red',key="CT_param",visible=False),sg.Tab('CTPET', layout_ctpet,title_color='Red',key="CTPET_param",visible=False),
                        sg.Tab('PET', layout_pet,title_color='Red',key="PET_param",visible=False),
                         sg.Tab('RM', layout_rm,title_color='Red',key="RM_param",visible=False)]], 
                       tab_location='centertop'), sg.Button('Close')]]  

window = sg.Window("Radiomic_app",tabgrp,size=(700,600),font=("Arial CE",12))

          
while True:
    event,values = window.read()
    
    if event == sg.WIN_CLOSED or event=="Exit":
        break
        
    if values["CT"]==True:
        CT = mod.count("CT")
        window["CT_param"].Update(visible=True)
        if CT==0:
            mod.append("CT")
    else:
        window["CT_param"].Update(visible=False)
        
    if values["CT/CT-PET"]==True:
        CTPET = mod.count("CT-PET")
        window["CTPET_param"].Update(visible=True)
        if CTPET==0:
            mod.append("CT-PET")
    else:
        window["CTPET_param"].Update(visible=False)
        
    if values["PET"]==True:
        PET = mod.count("PET")
        window["PET_param"].Update(visible=True)
        if PET==0:
            mod.append("PET")
    else:
        window["PET_param"].Update(visible=False)
        
    if values["MR"]==True:
        window["RM_param"].Update(visible=True)
        MR = mod.count("MR")
        if MR==0:
            mod.append("MR")
    else:
        window["RM_param"].Update(visible=False)
    
    
    
    if values["-IN-"]!='':
        window["-IN2-"].Update(values["-IN-"])
                 
    else:
        window["-IN2-"].Update("Choose a path")
    
    
    if event == 'Convert to NIFTI':
        if len(mod)==0:
            sg.popup_error('ERROR! Choose at least a modality!!')
            
        if values["-IN-"] == '':
            window["-IN2-"].Update("Choose a correct path")
            
        else:
            patient=os.listdir(values["-IN-"])
            prova = control_function(values["-IN-"],mod)
            if len(prova)==len(patient):
                yesno = sg.popup_yes_no("Your files already exists, do you want to continue?")
                if yesno=="yes":

                    dicom2nifti(values["-IN-"],mod)
                else:
                    window['PBAR_dicom'].update(current_count=100)
                    window['OUT_dicom'].update("Finish")
                    
            else:
                dicom2nifti(values["-IN-"],mod)
                
                
        window['PBAR_dicom'].update(current_count=100)
        window['OUT_dicom'].update("Finish")
        
############## CREATE CT PARAM FILE ############################################################################################
    
    if values["new_yaml_ct"]=="": 
        window["yaml_CT"].Update(visible=False)
        window["yaml_da_fuori_text_ct"].Update(visible=False)
        window["norm_text_ct"].Update(visible=False)
        window["norm_ct"].Update(visible=False)
        window["res_text_ct"].Update(visible=False)
        window["res_space_1_ct"].Update(visible=False)
        window["res_space_2_ct"].Update(visible=False)
        window["res_space_3_ct"].Update(visible=False)
        window["int_text_ct"].Update(visible=False)
        window["interp_ct"].Update(visible=False)
        window["precrop_text_ct"].Update(visible=False)
        window["precrop_ct"].Update(visible=False)
        window["reseg_text_ct"].Update(visible=False)
        window["res_range_ct"].Update(visible=False)
        window["bin_text_ct"].Update(visible=False)
        window["bin_w_ct"].Update(visible=False)
        window["force_text_ct"].Update(visible=False)
        window["force_ct"].Update(visible=False)
        window["choose_folder_text_ct"].Update(visible=False)
        window["-IN4-_ct"].Update(visible=False)
        window["generate_param_ct"].Update(visible=False) 
        
    elif values["new_yaml_ct"]=="no":
        
        window["yaml_da_fuori_text_ct"].Update(visible=True)
        window["yaml_CT"].Update(visible=True)
        window["yaml_CT_input"].Update(visible=True)
        window["yaml_CT_input"].Update(values["yaml_CT"])
      
        prova = yaml_da_fuori.count(values["yaml_CT"])
        if prova == 0:
            yaml_da_fuori.append(values["yaml_CT"])
    
        window["norm_text_ct"].Update(visible=False)
        window["norm_ct"].Update(visible=False)
        window["res_text_ct"].Update(visible=False)
        window["res_space_1_ct"].Update(visible=False)
        window["res_space_2_ct"].Update(visible=False)
        window["res_space_3_ct"].Update(visible=False)
        window["int_text_ct"].Update(visible=False)
        window["interp_ct"].Update(visible=False)
        window["precrop_text_ct"].Update(visible=False)
        window["precrop_ct"].Update(visible=False)
        window["reseg_text_ct"].Update(visible=False)
        window["res_range_ct"].Update(visible=False)
        window["bin_text_ct"].Update(visible=False)
        window["bin_w_ct"].Update(visible=False)
        window["force_text_ct"].Update(visible=False)
        window["force_ct"].Update(visible=False)
        window["choose_folder_text_ct"].Update(visible=False)
        window["-IN4-_ct"].Update(visible=False)
        window["generate_param_ct"].Update(visible=False) 
        
        
            
        
    else:
        window["yaml_da_fuori_text_ct"].Update(visible=False)
        window["norm_text_ct"].Update(visible=True)
        window["norm_ct"].Update(visible=True)
        window["res_text_ct"].Update(visible=True)
        window["res_space_1_ct"].Update(visible=True)
        window["res_space_2_ct"].Update(visible=True)
        window["res_space_3_ct"].Update(visible=True)
        window["int_text_ct"].Update(visible=True)
        window["interp_ct"].Update(visible=True)
        window["precrop_text_ct"].Update(visible=True)
        window["precrop_ct"].Update(visible=True)
        window["reseg_text_ct"].Update(visible=True)
        window["res_range_ct"].Update(visible=True)
        window["bin_text_ct"].Update(visible=True)
        window["bin_w_ct"].Update(visible=True)
        window["force_text_ct"].Update(visible=True)
        window["force_ct"].Update(visible=True)
        window["choose_folder_text_ct"].Update(visible=True)
        window["-IN4-_ct"].Update(visible=True)
        window["generate_param_ct"].Update(visible=True)
         
      
        
    if values["norm_ct"]==True:
        window["text_scale_ct"].Update(visible=True)
        window['scale_ct'].Update(visible=True)
        window["text out_ct"].Update(visible=True)
        window['outliers_ct'].Update(visible=True)
        if values["outliers_ct"]=="yes":
            window['out_scale_ct'].Update(visible=True)
        else:
            window['out_scale_ct'].Update(visible=False)
            #norm = "True"
    else:
        window["text_scale_ct"].Update(visible=False)
        window['scale_ct'].Update(visible=False)
        window["text out_ct"].Update(visible=False)
        window['outliers_ct'].Update(visible=False)
    
    if values["res_range_ct"]==True:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_ct"].Update(visible=True)
        window['lower_ct'].Update(visible=True)
        window['upper_text_ct'].Update(visible=True)
        window['upper_ct'].Update(visible=True)
        window['text_mode_ct'].Update(visible=True)
        window['res_mode_ct'].Update(visible=True)
    else:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_ct"].Update(visible=False)
        window['lower_ct'].Update(visible=False)
        window['upper_text_ct'].Update(visible=False)
        window['upper_ct'].Update(visible=False)
        window['text_mode_ct'].Update(visible=False)
        window['res_mode_ct'].Update(visible=False)
    

        
                
        
    if values["force_ct"]==True:
        window["force_dim_text_ct"].Update(visible=True)
        window["force_dim_ct"].Update(visible=True)
    else:
        window["force_dim_text_ct"].Update(visible=False)
        window["force_dim_ct"].Update(visible=False)
              
    if event == "generate_param_ct":
        yaml_name_ct=sg.popup_get_text('Enter parameter file name (ParamFile+Mod+.yaml)', title="Parameters File name")
        if values["norm_ct"]==True:
            norm1=True
            scale = float(values["scale_ct"])
            if values["outliers_ct"]=="yes":
                
                out_scale = float(values["out_scale_ct"])
        else:
            norm1=False
            scale = 1
        
            out_scale=1
        
        if values["res_range_ct"]==True:
            if values["upper_ct"]==0:
                res=float(values["lower_ct"])
            else:
                res=[float(values["lower_ct"]),float(values["upper_ct"])]
        else:
            res=None
   
        
        if values["res_mode_ct"]== "sigma (3std)":
            res_mode={"sigma":[-3,3]}
        else:
            res_mode = values["res_mode_ct"]
                        
                
        if values["force_ct"]==True:
            force = True
            force_dim = float(values["force_dim_ct"])
        else:
            force = False
            force_dim = 0
            
        pixel_spacing=[float(values["res_space_1_ct"]),float(values["res_space_2_ct"]),float(values["res_space_3_ct"])]
        save_path = values["-IN4-_ct"]+"/"+yaml_name_ct
        parameters_CT = yamlfile(norm1,scale,out_scale,pixel_spacing,values["interp_ct"],values["precrop_ct"],res,res_mode,float(values["bin_w_ct"]),force,force_dim)
        
        
        with open(save_path, 'w') as file:
            documents = yaml.dump(parameters_CT, file, default_flow_style=False)
            
        yn_ct = sg.popup_yes_no("Do you want to use this parameter file for radiomic?")
        if yn_ct=="Yes":
            yaml_da_fuori.append(save_path)
            
        elif yn_ct=="No":
            yN_ct = sg.popup_yes_no("Are you sure you want to upload another file?")
            if yN_ct=="Yes":
                new_CT_param_file = popup_get_file("Choose your param file")
                window["yaml_CT_input"].Update(visible=True)
                window["yaml_CT_input"].Update(new_CTPET_param_file)
    
                window["norm_text_ct"].Update(visible=False)
                window["norm_ct"].Update(visible=False)
                window["res_text_ct"].Update(visible=False)
                window["res_space_1_ct"].Update(visible=False)
                window["res_space_2_ct"].Update(visible=False)
                window["res_space_3_ct"].Update(visible=False)
                window["int_text_ct"].Update(visible=False)
                window["interp_ct"].Update(visible=False)
                window["precrop_text_ct"].Update(visible=False)
                window["precrop_ct"].Update(visible=False)
                window["reseg_text_ct"].Update(visible=False)
                window["res_range_ct"].Update(visible=False)
                window["bin_text_ct"].Update(visible=False)
                window["bin_w_ct"].Update(visible=False)
                window["force_text_ct"].Update(visible=False)
                window["force_ct"].Update(visible=False)
                window["choose_folder_text_ct"].Update(visible=False)
                window["-IN4-_ct"].Update(visible=False)
                window["generate_param_ct"].Update(visible=False) 
            
                yaml_da_fuori.append(new_CT_param_file)
                
            
            
########################## CTPET PARAM FILE ##################################################################################        
    if values["new_yaml_ctpet"]=="": 
        window["yaml_CTPET"].Update(visible=False)
        window["yaml_da_fuori_text_ctpet"].Update(visible=False)
        window["norm_text_ctpet"].Update(visible=False)
        window["norm_ctpet"].Update(visible=False)
        window["res_text_ctpet"].Update(visible=False)
        window["res_space_1_ctpet"].Update(visible=False)
        window["res_space_2_ctpet"].Update(visible=False)
        window["res_space_3_ctpet"].Update(visible=False)
        window["int_text_ctpet"].Update(visible=False)
        window["interp_ctpet"].Update(visible=False)
        window["precrop_text_ctpet"].Update(visible=False)
        window["precrop_ctpet"].Update(visible=False)
        window["reseg_text_ctpet"].Update(visible=False)
        window["res_range_ctpet"].Update(visible=False)
        window["bin_text_ctpet"].Update(visible=False)
        window["bin_w_ctpet"].Update(visible=False)
        window["force_text_ctpet"].Update(visible=False)
        window["force_ctpet"].Update(visible=False)
        window["choose_folder_text_ctpet"].Update(visible=False)
        window["-IN4-_ctpet"].Update(visible=False)
        window["generate_param_ctpet"].Update(visible=False) 
        
    elif values["new_yaml_ctpet"]=="no":
        
        window["yaml_da_fuori_text_ctpet"].Update(visible=True)
        window["yaml_CTPET"].Update(visible=True)
        window["yaml_CTPET_input"].Update(visible=True)
        window["yaml_CTPET_input"].Update(values["yaml_CTPET"])
        prova = yaml_da_fuori.count(values["yaml_CTPET_input"])
        if prova == 0:
            yaml_da_fuori.append(values["yaml_CTPET_input"])
    
        window["norm_text_ctpet"].Update(visible=False)
        window["norm_ctpet"].Update(visible=False)
        window["res_text_ctpet"].Update(visible=False)
        window["res_space_1_ctpet"].Update(visible=False)
        window["res_space_2_ctpet"].Update(visible=False)
        window["res_space_3_ctpet"].Update(visible=False)
        window["int_text_ctpet"].Update(visible=False)
        window["interp_ctpet"].Update(visible=False)
        window["precrop_text_ctpet"].Update(visible=False)
        window["precrop_ctpet"].Update(visible=False)
        window["reseg_text_ctpet"].Update(visible=False)
        window["res_range_ctpet"].Update(visible=False)
        window["bin_text_ctpet"].Update(visible=False)
        window["bin_w_ctpet"].Update(visible=False)
        window["force_text_ctpet"].Update(visible=False)
        window["force_ctpet"].Update(visible=False)
        window["choose_folder_text_ctpet"].Update(visible=False)
        window["-IN4-_ctpet"].Update(visible=False)
        window["generate_param_ctpet"].Update(visible=False) 
        
    else:
        window["yaml_da_fuori_text_ctpet"].Update(visible=False)
        window["norm_text_ctpet"].Update(visible=True)
        window["norm_ctpet"].Update(visible=True)
        window["res_text_ctpet"].Update(visible=True)
        window["res_space_1_ctpet"].Update(visible=True)
        window["res_space_2_ctpet"].Update(visible=True)
        window["res_space_3_ctpet"].Update(visible=True)
        window["int_text_ctpet"].Update(visible=True)
        window["interp_ctpet"].Update(visible=True)
        window["precrop_text_ctpet"].Update(visible=True)
        window["precrop_ctpet"].Update(visible=True)
        window["reseg_text_ctpet"].Update(visible=True)
        window["res_range_ctpet"].Update(visible=True)
        window["bin_text_ctpet"].Update(visible=True)
        window["bin_w_ctpet"].Update(visible=True)
        window["force_text_ctpet"].Update(visible=True)
        window["force_ctpet"].Update(visible=True)
        window["choose_folder_text_ctpet"].Update(visible=True)
        window["-IN4-_ctpet"].Update(visible=True)
        window["generate_param_ctpet"].Update(visible=True)  
        
    if values["norm_ctpet"]==True:
        window["text_scale_ctpet"].Update(visible=True)
        window['scale_ctpet'].Update(visible=True)
        window["text out_ctpet"].Update(visible=True)
        window['outliers_ctpet'].Update(visible=True)
        if values["outliers_ctpet"]=="yes":
            window['out_scale_ctpet'].Update(visible=True)
        else:
            window['out_scale_ctpet'].Update(visible=False)
            #norm = "True"
    else:
        window["text_scale_ctpet"].Update(visible=False)
        window['scale_ctpet'].Update(visible=False)
        window["text out_ctpet"].Update(visible=False)
        window['outliers_ctpet'].Update(visible=False)
    
    if values["res_range_ctpet"]==True:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_ctpet"].Update(visible=True)
        window['lower_ctpet'].Update(visible=True)
        window['upper_text_ctpet'].Update(visible=True)
        window['upper_ctpet'].Update(visible=True)
        window['text_mode_ctpet'].Update(visible=True)
        window['res_mode_ctpet'].Update(visible=True)
    else:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_ctpet"].Update(visible=False)
        window['lower_ctpet'].Update(visible=False)
        window['upper_text_ctpet'].Update(visible=False)
        window['upper_ctpet'].Update(visible=False)
        window['text_mode_ctpet'].Update(visible=False)
        window['res_mode_ctpet'].Update(visible=False)
    

        
                
        
    if values["force_ctpet"]==True:
        window["force_dim_text_ctpet"].Update(visible=True)
        window["force_dim_ctpet"].Update(visible=True)
    else:
        window["force_dim_text_ctpet"].Update(visible=False)
        window["force_dim_ctpet"].Update(visible=False)
              
    if event == "generate_param_ctpet":
        yaml_name_ctpet=sg.popup_get_text('Enter parameter file name (ParamFile+Mod+.yaml)', title="Parameters File name")
        if values["norm_ctpet"]==True:
            norm1=True
            scale = float(values["scale_ctpet"])
            if values["outliers_ctpet"]=="yes":
                
                out_scale = float(values["out_scale_ctpet"])
        else:
            norm1=False
            scale = 1
        
            out_scale=1
        
        if values["res_range_ctpet"]==True:
            if values["upper_ctpet"]==0:
                res=float(values["lower_ctpet"])
            else:
                res=[float(values["lower_ctpet"]),float(values["upper_ctpet"])]
        else:
            res=None
   
        
        if values["res_mode_ctpet"]== "sigma (3std)":
            res_mode={"sigma":[-3,3]}
        else:
            res_mode = values["res_mode_ctpet"]
                        
                
        if values["force_ctpet"]==True:
            force = True
            force_dim = float(values["force_dim_ctpet"])
        else:
            force = False
            force_dim = 0
            
        pixel_spacing=[float(values["res_space_1_ctpet"]),float(values["res_space_2_ctpet"]),float(values["res_space_3_ctpet"])]
        save_path = values["-IN4-_ctpet"]+"/"+yaml_name_ctpet
        parameters_CTPET = yamlfile(norm1,scale,out_scale,pixel_spacing,values["interp_ctpet"],values["precrop_ctpet"],res,res_mode,float(values["bin_w_ctpet"]),force,force_dim)
        
        with open(save_path, 'w') as file:
            documents = yaml.dump(parameters_CTPET, file, default_flow_style=False)
            
        yn_ctpet = sg.popup_yes_no("Do you want to use this parameter file for radiomic?")
        if yn_ctpet=="Yes":
            yaml_da_fuori.append(save_path)
            
        elif yn_ctpet=="No":
            yN_ctpet = sg.popup_yes_no("Are you sure you want to upload another file?")
            if yN_ctpet=="Yes":
                new_CTPET_param_file = popup_get_file("Choose your param file")
                window["yaml_CTPET_input"].Update(visible=True)
                window["yaml_CTPET_input"].Update(new_CTPET_param_file)
    
                window["norm_text_ctpet"].Update(visible=False)
                window["norm_ctpet"].Update(visible=False)
                window["res_text_ctpet"].Update(visible=False)
                window["res_space_1_ctpet"].Update(visible=False)
                window["res_space_2_ctpet"].Update(visible=False)
                window["res_space_3_ctpet"].Update(visible=False)
                window["int_text_ctpet"].Update(visible=False)
                window["interp_ctpet"].Update(visible=False)
                window["precrop_text_ctpet"].Update(visible=False)
                window["precrop_ctpet"].Update(visible=False)
                window["reseg_text_ctpet"].Update(visible=False)
                window["res_range_ctpet"].Update(visible=False)
                window["bin_text_ctpet"].Update(visible=False)
                window["bin_w_ctpet"].Update(visible=False)
                window["force_text_ctpet"].Update(visible=False)
                window["force_ctpet"].Update(visible=False)
                window["choose_folder_text_ctpet"].Update(visible=False)
                window["-IN4-_ctpet"].Update(visible=False)
                window["generate_param_ctpet"].Update(visible=False) 
            
                yaml_da_fuori.append(new_CTPET_param_file)
            
            
########################## PET PARAM FILE ##################################################################################        
    if values["new_yaml_pet"]=="": 
        window["yaml_da_fuori_text_pet"].Update(visible=False)
        window["norm_text_pet"].Update(visible=False)
        window["norm_pet"].Update(visible=False)
        window["res_text_pet"].Update(visible=False)
        window["res_space_1_pet"].Update(visible=False)
        window["res_space_2_pet"].Update(visible=False)
        window["res_space_3_pet"].Update(visible=False)
        window["int_text_pet"].Update(visible=False)
        window["interp_pet"].Update(visible=False)
        window["precrop_text_pet"].Update(visible=False)
        window["precrop_pet"].Update(visible=False)
        window["reseg_text_pet"].Update(visible=False)
        window["res_range_pet"].Update(visible=False)
        window["bin_text_pet"].Update(visible=False)
        window["bin_w_pet"].Update(visible=False)
        window["force_text_pet"].Update(visible=False)
        window["force_pet"].Update(visible=False)
        window["choose_folder_text_pet"].Update(visible=False)
        window["-IN4-_pet"].Update(visible=False)
        window["generate_param_pet"].Update(visible=False) 
        
    elif values["new_yaml_pet"]=="no":
        
        window["yaml_da_fuori_text_pet"].Update(visible=True)
        window["yaml_PET"].Update(visible=True)
        window["yaml_PET_input"].Update(visible=True)
        window["yaml_PET_input"].Update(values["yaml_PET"])
        prova = yaml_da_fuori.count(values["yaml_PET_input"])
        if prova == 0:
            yaml_da_fuori.append(values["yaml_PET_input"])
    
        window["norm_text_pet"].Update(visible=False)
        window["norm_pet"].Update(visible=False)
        window["res_text_pet"].Update(visible=False)
        window["res_space_1_pet"].Update(visible=False)
        window["res_space_2_pet"].Update(visible=False)
        window["res_space_3_pet"].Update(visible=False)
        window["int_text_pet"].Update(visible=False)
        window["interp_pet"].Update(visible=False)
        window["precrop_text_pet"].Update(visible=False)
        window["precrop_pet"].Update(visible=False)
        window["reseg_text_pet"].Update(visible=False)
        window["res_range_pet"].Update(visible=False)
        window["bin_text_pet"].Update(visible=False)
        window["bin_w_pet"].Update(visible=False)
        window["force_text_pet"].Update(visible=False)
        window["force_pet"].Update(visible=False)
        window["choose_folder_text_pet"].Update(visible=False)
        window["-IN4-_pet"].Update(visible=False)
        window["generate_param_pet"].Update(visible=False) 
        
    else:
        window["yaml_da_fuori_text_pet"].Update(visible=False)
        window["yaml_PET"].Update(visible=False)
        window["yaml_PET_input"].Update(visible=False)
        window["norm_text_pet"].Update(visible=True)
        window["norm_pet"].Update(visible=True)
        window["res_text_pet"].Update(visible=True)
        window["res_space_1_pet"].Update(visible=True)
        window["res_space_2_pet"].Update(visible=True)
        window["res_space_3_pet"].Update(visible=True)
        window["int_text_pet"].Update(visible=True)
        window["interp_pet"].Update(visible=True)
        window["precrop_text_pet"].Update(visible=True)
        window["precrop_pet"].Update(visible=True)
        window["reseg_text_pet"].Update(visible=True)
        window["res_range_pet"].Update(visible=True)
        window["bin_text_pet"].Update(visible=True)
        window["bin_w_pet"].Update(visible=True)
        window["force_text_pet"].Update(visible=True)
        window["force_pet"].Update(visible=True)
        window["choose_folder_text_pet"].Update(visible=True)
        window["-IN4-_pet"].Update(visible=True)
        window["generate_param_pet"].Update(visible=True)  
        
        
    if values["norm_pet"]==True:
        window["text_scale_pet"].Update(visible=True)
        window['scale_pet'].Update(visible=True)
        window["text out_pet"].Update(visible=True)
        window['outliers_pet'].Update(visible=True)
        if values["outliers_pet"]=="yes":
            window['out_scale_pet'].Update(visible=True)
        else:
            window['out_scale_pet'].Update(visible=False)
            #norm = "True"
    else:
        window["text_scale_pet"].Update(visible=False)
        window['scale_pet'].Update(visible=False)
        window["text out_pet"].Update(visible=False)
        window['outliers_pet'].Update(visible=False)
    
    if values["res_range_pet"]==True:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_pet"].Update(visible=True)
        window['lower_pet'].Update(visible=True)
        window['upper_text_pet'].Update(visible=True)
        window['upper_pet'].Update(visible=True)
        window['text_mode_pet'].Update(visible=True)
        window['res_mode_pet'].Update(visible=True)
    else:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_pet"].Update(visible=False)
        window['lower_pet'].Update(visible=False)
        window['upper_text_pet'].Update(visible=False)
        window['upper_pet'].Update(visible=False)
        window['text_mode_pet'].Update(visible=False)
        window['res_mode_pet'].Update(visible=False)
    

        
                
        
    if values["force_pet"]==True:
        window["force_dim_text_pet"].Update(visible=True)
        window["force_dim_pet"].Update(visible=True)
    else:
        window["force_dim_text_pet"].Update(visible=False)
        window["force_dim_pet"].Update(visible=False)
              
    if event == "generate_param_pet":
        yaml_name_pet=sg.popup_get_text('Enter parameter file name (ParamFile+Mod+.yaml)', title="Parameters File name")
        if values["norm_pet"]==True:
            norm1=True
            scale = float(values["scale_pet"])
            if values["outliers_pet"]=="yes":
                
                out_scale = float(values["out_scale_pet"])
        else:
            norm1=False
            scale = 1
        
            out_scale=1
        
        if values["res_range_pet"]==True:
            if values["upper_pet"]==0:
                res=float(values["lower_pet"])
            else:
                res=[float(values["lower_pet"]),float(values["upper_pet"])]
        else:
            res=None
   
        
        if values["res_mode_pet"]== "sigma (3std)":
            res_mode={"sigma":[-3,3]}
        else:
            res_mode = values["res_mode_pet"]
                        
                
        if values["force_pet"]==True:
            force = True
            force_dim = float(values["force_dim_pet"])
        else:
            force = False
            force_dim = 0
            
        pixel_spacing=[float(values["res_space_1_pet"]),float(values["res_space_2_pet"]),float(values["res_space_3_pet"])]
        save_path = values["-IN4-_pet"]+"/"+yaml_name_pet
        parameters_PET = yamlfile(norm1,scale,out_scale,pixel_spacing,values["interp_pet"],values["precrop_pet"],res,res_mode,float(values["bin_w_pet"]),force,force_dim)
        with open(save_path, 'w') as file:
            documents = yaml.dump(parameters_PET, file, default_flow_style=False)
         
        yn_pet = sg.popup_yes_no("Do you want to use this parameter file for radiomic?")
        if yn_pet=="Yes":
            yaml_da_fuori.append(save_path)
            
        elif yn_pet=="No":
            yN_pet = sg.popup_yes_no("Are you sure you want to upload another file?")
            if yN_pet=="Yes":
                new_PET_param_file = popup_get_file("Choose your param file")
                window["yaml_PET_input"].Update(visible=True)
                window["yaml_PET_input"].Update(new_PET_param_file)
    
                window["norm_text_pet"].Update(visible=False)
                window["norm_pet"].Update(visible=False)
                window["res_text_pet"].Update(visible=False)
                window["res_space_1_pet"].Update(visible=False)
                window["res_space_2_pet"].Update(visible=False)
                window["res_space_3_pet"].Update(visible=False)
                window["int_text_pet"].Update(visible=False)
                window["interp_pet"].Update(visible=False)
                window["precrop_text_pet"].Update(visible=False)
                window["precrop_pet"].Update(visible=False)
                window["reseg_text_pet"].Update(visible=False)
                window["res_range_pet"].Update(visible=False)
                window["bin_text_pet"].Update(visible=False)
                window["bin_w_pet"].Update(visible=False)
                window["force_text_pet"].Update(visible=False)
                window["force_pet"].Update(visible=False)
                window["choose_folder_text_pet"].Update(visible=False)
                window["-IN4-_pet"].Update(visible=False)
                window["generate_param_pet"].Update(visible=False) 
            
                yaml_da_fuori.append(new_PET_param_file)
                
########################## RM PARAM FILE ##################################################################################        
    if values["new_yaml_rm"]=="": 
        window["yaml_da_fuori_text_rm"].Update(visible=False)
        window["norm_text_rm"].Update(visible=False)
        window["norm_rm"].Update(visible=False)
        window["res_text_rm"].Update(visible=False)
        window["res_space_1_rm"].Update(visible=False)
        window["res_space_2_rm"].Update(visible=False)
        window["res_space_3_rm"].Update(visible=False)
        window["int_text_rm"].Update(visible=False)
        window["interp_rm"].Update(visible=False)
        window["precrop_text_rm"].Update(visible=False)
        window["precrop_rm"].Update(visible=False)
        window["reseg_text_rm"].Update(visible=False)
        window["res_range_rm"].Update(visible=False)
        window["bin_text_rm"].Update(visible=False)
        window["bin_w_rm"].Update(visible=False)
        window["force_text_rm"].Update(visible=False)
        window["force_rm"].Update(visible=False)
        window["choose_folder_text_rm"].Update(visible=False)
        window["-IN4-_rm"].Update(visible=False)
        window["generate_param_rm"].Update(visible=False) 
        
    elif values["new_yaml_rm"]=="no":
        
        window["yaml_da_fuori_text_rm"].Update(visible=True)
        window["yaml_RM"].Update(visible=True)
        window["yaml_RM_input"].Update(visible=True)
        window["yaml_RM_input"].Update(values["yaml_RM"])
        prova = yaml_da_fuori.count(values["yaml_RM_input"])
        if prova == 0:
            yaml_da_fuori.append(values["yaml_RM_input"])
    
        window["norm_text_rm"].Update(visible=False)
        window["norm_rm"].Update(visible=False)
        window["res_text_rm"].Update(visible=False)
        window["res_space_1_rm"].Update(visible=False)
        window["res_space_2_rm"].Update(visible=False)
        window["res_space_3_rm"].Update(visible=False)
        window["int_text_rm"].Update(visible=False)
        window["interp_rm"].Update(visible=False)
        window["precrop_text_rm"].Update(visible=False)
        window["precrop_rm"].Update(visible=False)
        window["reseg_text_rm"].Update(visible=False)
        window["res_range_rm"].Update(visible=False)
        window["bin_text_rm"].Update(visible=False)
        window["bin_w_rm"].Update(visible=False)
        window["force_text_rm"].Update(visible=False)
        window["force_rm"].Update(visible=False)
        window["choose_folder_text_rm"].Update(visible=False)
        window["-IN4-_rm"].Update(visible=False)
        window["generate_param_rm"].Update(visible=False) 
        
    else:
        window["yaml_da_fuori_text_rm"].Update(visible=False)
        window["yaml_RM"].Update(visible=False)
        window["yaml_RM_input"].Update(visible=False)
        window["norm_text_rm"].Update(visible=True)
        window["norm_rm"].Update(visible=True)
        window["res_text_pet"].Update(visible=True)
        window["res_space_1_rm"].Update(visible=True)
        window["res_space_2_rm"].Update(visible=True)
        window["res_space_3_rm"].Update(visible=True)
        window["int_text_rm"].Update(visible=True)
        window["interp_rm"].Update(visible=True)
        window["precrop_text_rm"].Update(visible=True)
        window["precrop_rm"].Update(visible=True)
        window["reseg_text_rm"].Update(visible=True)
        window["res_range_rm"].Update(visible=True)
        window["bin_text_rm"].Update(visible=True)
        window["bin_w_rm"].Update(visible=True)
        window["force_text_rm"].Update(visible=True)
        window["force_rm"].Update(visible=True)
        window["choose_folder_text_rm"].Update(visible=True)
        window["-IN4-_rm"].Update(visible=True)
        window["generate_param_rm"].Update(visible=True)  
        
        
    if values["norm_rm"]==True:
        window["text_scale_rm"].Update(visible=True)
        window['scale_rm'].Update(visible=True)
        window["text out_rm"].Update(visible=True)
        window['outliers_rm'].Update(visible=True)
        if values["outliers_rm"]=="yes":
            window['out_scale_rm'].Update(visible=True)
        else:
            window['out_scale_rm'].Update(visible=False)
            #norm = "True"
    else:
        window["text_scale_rm"].Update(visible=False)
        window['scale_rm'].Update(visible=False)
        window["text out_rm"].Update(visible=False)
        window['outliers_rm'].Update(visible=False)
    
    if values["res_range_rm"]==True:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_rm"].Update(visible=True)
        window['lower_rm'].Update(visible=True)
        window['upper_text_rm'].Update(visible=True)
        window['upper_rm'].Update(visible=True)
        window['text_mode_rm'].Update(visible=True)
        window['res_mode_pet'].Update(visible=True)
    else:
        #window["res_range_values"].Update(visible=True)
        window["lower_text_rm"].Update(visible=False)
        window['lower_rm'].Update(visible=False)
        window['upper_text_rm'].Update(visible=False)
        window['upper_rm'].Update(visible=False)
        window['text_mode_rm'].Update(visible=False)
        window['res_mode_rm'].Update(visible=False)
    

        
                
        
    if values["force_rm"]==True:
        window["force_dim_text_rm"].Update(visible=True)
        window["force_dim_rm"].Update(visible=True)
    else:
        window["force_dim_text_rm"].Update(visible=False)
        window["force_dim_rm"].Update(visible=False)
              
    if event == "generate_param_rm":
        yaml_name_rm=sg.popup_get_text('Enter parameter file name (ParamFile+Mod+.yaml)', title="Parameters File name")
        if values["norm_rm"]==True:
            norm1=True
            scale = float(values["scale_rm"])
            if values["outliers_rm"]=="yes":
                
                out_scale = float(values["out_scale_rm"])
        else:
            norm1=False
            scale = 1
        
            out_scale=1
        
        if values["res_range_rm"]==True:
            if values["upper_rm"]==0:
                res=float(values["lower_rm"])
            else:
                res=[float(values["lower_rm"]),float(values["upper_rm"])]
        else:
            res=None
   
        
        if values["res_mode_rm"]== "sigma (3std)":
            res_mode={"sigma":[-3,3]}
        else:
            res_mode = values["res_mode_rm"]
                        
                
        if values["force_rm"]==True:
            force = True
            force_dim = float(values["force_dim_rm"])
        else:
            force = False
            force_dim = 0
            
        pixel_spacing=[float(values["res_space_1_rm"]),float(values["res_space_2_rm"]),float(values["res_space_3_rm"])]
        save_path = values["-IN4-_rm"]+"/"+yaml_name_rm
        parameters_RM = yamlfile(norm1,scale,out_scale,pixel_spacing,values["interp_rm"],values["precrop_rm"],res,res_mode,float(values["bin_w_rm"]),force,force_dim)
        with open(save_path, 'w') as file:
            documents = yaml.dump(parameters_PET, file, default_flow_style=False)
         
        yn_rm = sg.popup_yes_no("Do you want to use this parameter file for radiomic?")
        if yn_rm=="Yes":
            yaml_da_fuori.append(save_path)
            
        elif yn_rm=="No":
            yN_rm = sg.popup_yes_no("Are you sure you want to upload another file?")
            if yN_rm=="Yes":
                new_RM_param_file = popup_get_file("Choose your param file")
                window["yaml_RM_input"].Update(visible=True)
                window["yaml_RM_input"].Update(new_RM_param_file)
    
                window["norm_text_rm"].Update(visible=False)
                window["norm_rm"].Update(visible=False)
                window["res_text_rm"].Update(visible=False)
                window["res_space_1_rm"].Update(visible=False)
                window["res_space_2_rm"].Update(visible=False)
                window["res_space_3_rm"].Update(visible=False)
                window["int_text_rm"].Update(visible=False)
                window["interp_rm"].Update(visible=False)
                window["precrop_text_rm"].Update(visible=False)
                window["precrop_rm"].Update(visible=False)
                window["reseg_text_rm"].Update(visible=False)
                window["res_range_rm"].Update(visible=False)
                window["bin_text_rm"].Update(visible=False)
                window["bin_w_rm"].Update(visible=False)
                window["force_text_rm"].Update(visible=False)
                window["force_rm"].Update(visible=False)
                window["choose_folder_text_rm"].Update(visible=False)
                window["-IN4-_rm"].Update(visible=False)
                window["generate_param_rm"].Update(visible=False) 
            
                yaml_da_fuori.append(new_RM_param_file)
        

                                            
############## RADIOMIC ###################################################################################################
    if event == "radiomic_proceed":
        
        new_list = [x for x in yaml_da_fuori if x != '']
       
        if len(new_list)!=len(mod):
            error_parameters = sg.popup_error("Parameter files are missing")
            
        elif values["-IN2-"]=="Choose a path":
            error_patient = sg.popup_error("Choose patient folder")
            
        else:
            window["text_radiomic_save"].Update(visible=True)
            window["input_radiomic_save"].Update(visible=True)
            window["radiomic_save"].Update(visible=True)
            window["radiomic"].Update(visible=True)
            window['PBAR'].Update(visible=True)
            window['-OUT-'].Update(visible=True)
            
        
    if event=="radiomic":
 
        radiomic_name=sg.popup_get_text('Enter radiomic file name', title="Radiomic File name")
        if radiomic_name=="":
            sg.popup_error("Choose a Radiomic file name before proceding")
        else:
            if values["CT"]==True:
                modality="CT"
                if values["new_yaml_ct"]=="no" or (yn_ct=="No" and yN_ct=="Yes"):
                    param_file = values["yaml_CT"]
                    radiomic_file_CT = radiomics(param_file,values["-IN-"],modality)

                elif values["new_yaml_ct"]=="sì" or yn_ct=="Yes":
                    param_file = parameters_CT
                    radiomic_file_CT = radiomics(param_file,values["-IN-"],modality)
                    window["yaml_da_fuori_text_ct"].Update(visible=False)
                    window["yaml_CT"].Update(visible=False)
                    window["yaml_CT_input"].Update(visible=False)

            if values["CT/CT-PET"]==True:
                modality="CT-PET"
                if values["new_yaml_ctpet"]=="no" or (yn_ctpet=="No" and yN_ctpet=="Yes"):
                    param_file = values["yaml_CTPET"]
                    radiomic_file_CTPET = radiomics(param_file,values["-IN-"],modality)

                elif values["new_yaml_ctpet"]=="sì" or yn_ctpet=="Yes":
                    param_file = parameters_CTPET
                    radiomic_file_CTPET = radiomics(param_file,values["-IN-"],modality)

            if values["PET"]==True:
                modality="PET"
                if values["new_yaml_pet"]=="no" or (yn_pet=="No" and yN_pet=="Yes"):
                    param_file = values["yaml_PET"]
                    radiomic_file_PET = radiomics(param_file,values["-IN-"],modality)

                elif values["new_yaml_pet"]=="sì" or yn_pet=="Yes":
                    param_file = parameters_PET
                    radiomic_file_PET = radiomics(param_file,values["-IN-"],modality)

            if values["MR"]==True:
                modality="RM"
                if values["new_yaml_rm"]=="no" or (yn_rm=="No" and yN_rm=="Yes"):
                    param_file = values["yaml_RM"]
                    radiomic_file_RM = radiomics(param_file,values["-IN-"],modality)

                elif values["new_yaml_rm"]=="sì" or yn_rm=="Yes":
                    param_file = parameters_RM
                    radiomic_file_RM = radiomics(param_file,values["-IN-"],modality)

            df_final = []
            if values["CT"]==True:
                df_final.append(radiomic_file_CT)
            if values["CT/CT-PET"]==True:
                df_final.append(radiomic_file_CTPET)
            if values["PET"]==True:
                df_final.append(radiomic_file_PET)
            if values["MR"]==True:
                df_final.append(radiomic_file_RM)


            result = pd.concat(df_final)       
            result.to_excel(values["radiomic_save"]+"/"+radiomic_name+".xlsx",index=False)

            window["radiomic_status"].Update("Finish")
            window['-OUT-'].update("Finish") 

            window["reset"].Update(visible=True)
        
    if event=="reset":
        window["-IN-"].Update("")
        window["-IN2-"].Update("")
        window["CT"].Update(False)
        window["CT"].Update(False)
        window["CT"].Update(False)
        window["CT"].Update(False)
        window["CT_param"].Update(visible=False)
        window["CTPET_param"].Update(visible=False)
        window["PET_param"].Update(visible=False)
        window["RM_param"].Update(visible=False)
        window["text_radiomic_save"].Update(visible=False)
        window["input_radiomic_save"].Update(visible=False)
        window["radiomic_save"].Update(visible=False)
        window["text_radiomic_save"].Update("")
        window["input_radiomic_save"].Update("")
        window["radiomic_proceed"].Update(visible=False)
        window["text_radiomic_save"].Update(visible=False)
        window["radiomic_save"].Update("")
        window["radiomic"].Update(visible=False)
        window['PBAR'].Update(visible=False)
        window['-OUT-'].Update(visible=False)
        window['PBAR'].Update(visible="")
        window['-OUT-'].Update(visible="")